In [1]:
from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

# Tracing

A way to see what the agents are doing.

In [1]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)

# when doing a single run the tracing is automatically done
result = await Runner.run(agent, "Hello! How are you?")
result.final_output

'HELLO! I’M DOING GREAT, THANK YOU! HOW ABOUT YOU?'

In [2]:
from agents import trace

joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
)
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in a different language.",
)

with trace("Joke Translation Workflow"):
    joke_result = await Runner.run(joke_agent, "Cheese")
    translated_result = await Runner.run(language_agent, f"Translate this joke to Spanish: {joke_result.final_output}")
    print(f"Translated joke:\n{translated_result.final_output}")

Translated joke:
¿Por qué el queso se sentó solo en el almuerzo?

¡Porque se sentía azul!


In [3]:
from agents import function_tool, RunContextWrapper, handoff
from pydantic import BaseModel

class ManagerEscalation(BaseModel):
    issue: str # the issue being escalated
    why: str # why can you not handle it? Used for training in the future

@function_tool
def create_ticket(issue: str):
    """"
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID: 12345"
    # In a real-world scenario, this would interact with a ticketing system

manager_agent = Agent(
    name="Manager",
    handoff_description="Handles escalated issues that require managerial attention",
    instructions=(
        "You handle escalated customer issues that the initial custom service agent could not resolve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."
    ),
    tools=[create_ticket],
)

def on_manager_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
    print("Escalating to manager agent: ", input.issue)
    print("Reason for escalation: ", input.why)

    # here we might store the escalation in a database or log it for future reference

customer_service_agent = Agent(
    name="Customer Service",
    instructions="You assist customers with general inquiries and basic troubleshooting. " +
                 "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue yourself.",
    handoffs=[handoff(
        agent=manager_agent,
        input_type=ManagerEscalation,
        on_handoff=on_manager_handoff,
    )]
)

with trace("Customer Service Hotline"):
    result = await Runner.run(customer_service_agent, "Hello im having an issue with refunding my order. The page is blank and I cannot see anything. I have tried clearing my cache and using a different browser. I am not sure what else to do.")
    print(result.final_output)

Escalating to manager agent:  Customer unable to access the refund page as it appears blank despite clearing cache and trying different browsers.
Reason for escalation:  The issue involves a potentially complex web page malfunction or backend issue affecting the refund portal, which cannot be resolved through basic troubleshooting steps.
Creating ticket for issue: Customer unable to access the refund page as it appears blank despite clearing cache and trying different browsers.
I've created a ticket for this issue (Ticket ID: 12345), and our technical team will look into it as soon as possible. We'll update you once it's resolved. Thank you for your patience!
